# 10 — Bayesian Statistics
**Author:** Ebenezer Adjartey

Covers: Bayes' theorem, conjugate priors, Bayesian linear regression, MCMC (Metropolis-Hastings), credible intervals vs confidence intervals, Bayes Factor.

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import beta, norm, binom
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)
sns.set_theme(style='whitegrid')
print('Libraries loaded.')

## 1. Bayes Theorem Illustration

In [ ]:
# Medical test example
# Disease prevalence = 1%, Test sensitivity=95%, Specificity=90%
P_disease  = 0.01     # prior
P_pos_d    = 0.95     # P(+|disease)
P_pos_nd   = 0.10     # P(+|no disease)

P_no_disease = 1 - P_disease
P_pos = P_pos_d * P_disease + P_pos_nd * P_no_disease  # total probability

# Posterior: P(disease|+)
P_disease_pos = (P_pos_d * P_disease) / P_pos

print('=== BAYES THEOREM: Medical Test ===')
print(f'P(disease)         = {P_disease:.3f}  (prior)')
print(f'P(+|disease)       = {P_pos_d:.3f}  (sensitivity)')
print(f'P(+|no disease)    = {P_pos_nd:.3f}  (false positive rate)')
print(f'P(+)               = {P_pos:.4f}  (marginal likelihood)')
print(f'P(disease|+)       = {P_disease_pos:.4f}  (posterior)')
print(f'\nDespite 95% sensitivity, only {P_disease_pos*100:.1f}% of positives actually have disease')
print('This is the base-rate fallacy / Bayesian updating in action')

## 2. Beta-Binomial Conjugate Prior (Bayesian Inference for Proportion)

In [ ]:
# Prior: Beta(alpha=2, beta=2) - weak prior centered at 0.5
# Data:  12 successes in 20 trials
alpha_prior, beta_prior = 2, 2
successes, n_trials = 12, 20

# Posterior: Beta(alpha + successes, beta + failures)
alpha_post = alpha_prior + successes
beta_post  = beta_prior + (n_trials - successes)

theta = np.linspace(0, 1, 300)
prior_pdf     = beta.pdf(theta, alpha_prior, beta_prior)
likelihood    = binom.pmf(successes, n_trials, theta)  # unnormalized
likelihood_n  = likelihood / np.trapz(likelihood, theta)  # normalize
posterior_pdf = beta.pdf(theta, alpha_post, beta_post)

# Posterior summaries
post_mean   = alpha_post / (alpha_post + beta_post)
post_mode   = (alpha_post-1) / (alpha_post + beta_post - 2)
ci_low, ci_high = beta.ppf(0.025, alpha_post, beta_post), beta.ppf(0.975, alpha_post, beta_post)

print(f'Prior: Beta({alpha_prior},{beta_prior})  mean={alpha_prior/(alpha_prior+beta_prior):.3f}')
print(f'Data:  {successes}/{n_trials} successes  MLE={successes/n_trials:.3f}')
print(f'Posterior: Beta({alpha_post},{beta_post})')
print(f'  Mean  = {post_mean:.4f}')
print(f'  Mode  = {post_mode:.4f}')
print(f'  95% Credible Interval: ({ci_low:.4f}, {ci_high:.4f})')

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(theta, prior_pdf,     'b--', lw=2, label=f'Prior Beta({alpha_prior},{beta_prior})')
ax.plot(theta, likelihood_n,  'g:',  lw=2, label=f'Likelihood (normalized)')
ax.plot(theta, posterior_pdf, 'r-',  lw=2, label=f'Posterior Beta({alpha_post},{beta_post})')
ax.axvline(ci_low,  color='orange', linestyle=':', lw=1.5)
ax.axvline(ci_high, color='orange', linestyle=':', lw=1.5, label='95% Credible Interval')
ax.set_title('Bayesian Updating: Prior → Posterior')
ax.set_xlabel('theta (proportion)'); ax.set_ylabel('Density')
ax.legend()
plt.tight_layout()
os.makedirs('10_bayesian_statistics', exist_ok=True)
plt.savefig('10_bayesian_statistics/bayesian_updating.png', dpi=100, bbox_inches='tight')
plt.show(); print('Saved.')

## 3. Normal-Normal Conjugate (Bayesian Inference for Mean)

In [ ]:
# Known variance, unknown mean
# Prior: mu ~ N(mu_0, tau^2) = N(5, 4)
mu_0, tau2 = 5.0, 4.0
sigma2 = 9.0      # known population variance
data   = np.array([7.2, 6.8, 8.1, 7.5, 6.9, 8.3, 7.0, 7.8])
n      = len(data)
x_bar  = data.mean()

# Posterior: N(mu_n, tau_n^2)
tau_n2  = 1 / (1/tau2 + n/sigma2)
mu_n    = tau_n2 * (mu_0/tau2 + n*x_bar/sigma2)

ci_low  = norm.ppf(0.025, mu_n, np.sqrt(tau_n2))
ci_high = norm.ppf(0.975, mu_n, np.sqrt(tau_n2))

print(f'Prior: N(mu_0={mu_0}, sigma^2={tau2})')
print(f'Data:  n={n}, x_bar={x_bar:.4f}')
print(f'Posterior: N(mu_n={mu_n:.4f}, sigma_n^2={tau_n2:.4f})')
print(f'95% Credible Interval: ({ci_low:.4f}, {ci_high:.4f})')

# Compare frequentist CI
freq_ci = norm.interval(0.95, loc=x_bar, scale=np.sqrt(sigma2/n))
print(f'\nFrequentist 95% CI: ({freq_ci[0]:.4f}, {freq_ci[1]:.4f})')
print('Bayesian CI is pulled toward prior; frequentist CI centers on sample mean')

## 4. MCMC: Metropolis-Hastings Algorithm

In [ ]:
# Estimate posterior of mu in N(mu, sigma=2) model with N(0,10) prior
# Data: 50 observations from N(3, 2)
true_mu = 3.0
sigma   = 2.0
data_mc = np.random.normal(true_mu, sigma, 50)

def log_posterior(mu, data, sigma=2.0, mu_prior=0, sigma_prior=10):
    log_likelihood = norm.logpdf(data, mu, sigma).sum()
    log_prior      = norm.logpdf(mu, mu_prior, sigma_prior)
    return log_likelihood + log_prior

# Metropolis-Hastings
n_iter     = 10000
proposal_sd = 0.2
chain       = np.zeros(n_iter)
chain[0]    = 0.0   # starting value
accepted    = 0

for i in range(1, n_iter):
    proposal = chain[i-1] + np.random.normal(0, proposal_sd)
    log_ratio = log_posterior(proposal, data_mc) - log_posterior(chain[i-1], data_mc)
    if np.log(np.random.uniform()) < log_ratio:
        chain[i] = proposal
        accepted += 1
    else:
        chain[i] = chain[i-1]

burn_in  = 2000
samples  = chain[burn_in:]
acc_rate = accepted / n_iter

print(f'MCMC Metropolis-Hastings: {n_iter} iterations, burn-in={burn_in}')
print(f'Acceptance rate: {acc_rate:.3f}  (ideal: 0.20-0.50)')
print(f'Posterior mean: {samples.mean():.4f}  (true: {true_mu})')
print(f'Posterior SD:   {samples.std():.4f}')
ci = np.percentile(samples, [2.5, 97.5])
print(f'95% Credible Interval: ({ci[0]:.4f}, {ci[1]:.4f})')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Trace plot
axes[0].plot(chain[:500], lw=.5, alpha=.8)
axes[0].axvline(burn_in, color='red', linestyle='--', label='Burn-in')
axes[0].set_title('MCMC Trace Plot'); axes[0].set_xlabel('Iteration')
axes[0].legend(fontsize=8)

# Posterior histogram
axes[1].hist(samples, bins=50, density=True, color='steelblue', alpha=.7)
x_range = np.linspace(samples.min(), samples.max(), 200)
axes[1].plot(x_range, norm.pdf(x_range, samples.mean(), samples.std()), 'r-', lw=2)
axes[1].axvline(true_mu, color='green', linestyle='--', label=f'True mu={true_mu}')
axes[1].set_title('Posterior Distribution'); axes[1].legend(fontsize=8)

# Autocorrelation of chain
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(samples, lags=40, ax=axes[2], title='Chain Autocorrelation')

plt.tight_layout()
plt.savefig('10_bayesian_statistics/mcmc_diagnostics.png', dpi=100, bbox_inches='tight')
plt.show(); print('Saved.')

## 5. Bayesian vs Frequentist: Credible vs Confidence Intervals

In [ ]:
print('=== Credible Interval vs Confidence Interval ===')
print()
print('Frequentist 95% CI:')
print('  "If we repeat this experiment many times, 95% of such intervals will contain the true mu."')
print('  The true mu is FIXED; randomness is in the interval.')
print()
print('Bayesian 95% Credible Interval:')
print('  "There is 95% probability that mu lies in this interval, given the data."')
print('  mu is treated as a RANDOM VARIABLE with a distribution.')
print()

# Numerical comparison
n_exp = 100
true_mu_comp = 5.0
sigma_comp   = 3.0

freq_captures, bayes_captures = 0, 0
for _ in range(n_exp):
    data_e = np.random.normal(true_mu_comp, sigma_comp, 30)
    x_bar_e = data_e.mean()
    se = sigma_comp / np.sqrt(len(data_e))
    freq_low, freq_high = x_bar_e - 1.96*se, x_bar_e + 1.96*se
    if freq_low <= true_mu_comp <= freq_high:
        freq_captures += 1

print(f'Simulation: {n_exp} experiments')
print(f'Frequentist 95% CI coverage: {freq_captures}/{n_exp} = {freq_captures}%')

## 6. Bayes Factor

In [ ]:
# Bayes Factor: H0: p=0.5 vs H1: p~Beta(1,1)=Uniform(0,1)
# Data: 15 successes in 20 trials
k, n_bf = 15, 20
p_H0 = 0.5

from scipy.special import comb
from scipy.integrate import quad

# Marginal likelihood under H0
marginal_H0 = binom.pmf(k, n_bf, p_H0)

# Marginal likelihood under H1: integrate over uniform prior
integrand = lambda p: binom.pmf(k, n_bf, p) * 1  # Beta(1,1) = Uniform
marginal_H1, _ = quad(integrand, 0, 1)

BF_10 = marginal_H1 / marginal_H0
BF_01 = 1 / BF_10

print(f'k={k}, n={n_bf} (MLE p_hat={k/n_bf:.2f})')
print(f'Marginal likelihood H0 (p=0.5): {marginal_H0:.6f}')
print(f'Marginal likelihood H1 (p~U):   {marginal_H1:.6f}')
print(f'Bayes Factor BF_10 = {BF_10:.4f}  (H1 vs H0)')
print(f'Bayes Factor BF_01 = {BF_01:.4f}  (H0 vs H1)')
print()
print('Jeffreys scale:')
print('  BF>100: Decisive evidence; 10-100: Strong; 3-10: Moderate; 1-3: Anecdotal')
if BF_10 > 10:
    print(f'  BF={BF_10:.2f}: Strong evidence FOR H1')
elif BF_10 > 3:
    print(f'  BF={BF_10:.2f}: Moderate evidence FOR H1')
else:
    print(f'  BF={BF_10:.2f}: Weak/Anecdotal evidence')

## Key Takeaways

- **Bayes' theorem**: posterior ∝ likelihood × prior
- **Conjugate priors**: analytical posterior (Beta-Binomial, Normal-Normal)
- **MCMC**: samples from intractable posteriors; monitor trace plots and acceptance rate
- **Credible interval**: direct probability statement about the parameter
- **Bayes Factor**: compares models without p-values; sensitive to prior choice
